In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import * 

In [0]:
spark = SparkSession.builder.appName("ecom_data_pipeline").getOrCreate()

## locations

users = /mnt/delta/tables/bronze/users  

sellers = /mnt/delta/tables/bronze/sellers 

buyers = /mnt/delta/tables/bronze/buyers 

countries = /mnt/delta/tables/bronze/countries

### Read **Users** Delta Table

In [0]:
usersDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/users")

## Transformations

In [0]:
# Normalize country codes to uppercase
usersDF = usersDF.withColumn("countrycode", upper(col("countrycode")))

In [0]:
# Handling multiple languages elegantly with `expr` and `case when`
usersDF = usersDF.withColumn("language_full", 
                             expr("CASE WHEN language = 'EN' THEN 'English' " +
                                  "WHEN language = 'FR' THEN 'French' " +
                                  "ELSE 'Other' END"))

In [0]:
# Correcting potential data entry errors in `gender` column
usersDF = usersDF.withColumn("gender", 
                            when(col("gender").startswith("M"), "Male")
                            .when(col("gender").startswith("F"), "Female")
                            .otherwise("Other"))

In [0]:
# Using `regexp_replace` to clean `civilitytitle` values
usersDF = usersDF.withColumn("civilitytitle_clean", 
                             regexp_replace("civilitytitle", "(Mme|Ms|Mrs)", "Ms"))

In [0]:
# Derive new column `years_since_last_login` from `dayssincelastlogin`
usersDF = usersDF.withColumn("years_since_last_login", col("dayssincelastlogin") / 365)

In [0]:
# Calculate age of account in years and categorize into `account_age_group`
usersDF = usersDF.withColumn("account_age_years", round(col("seniority") / 365, 2))
usersDF = usersDF.withColumn("account_age_group",
                             when(col("account_age_years") < 1, "New")
                             .when((col("account_age_years") >= 1) & (col("account_age_years") < 3), "Intermediate")
                             .otherwise("Experienced"))

In [0]:
# Add a column with the current year for comparison
usersDF = usersDF.withColumn("current_year", year(current_date()))

In [0]:
# Creatively combining strings to form a unique user descriptor
usersDF = usersDF.withColumn("user_descriptor", 
                             concat(col("gender"), lit("_"), 
                                    col("countrycode"), lit("_"), 
                                    expr("substring(civilitytitle_clean, 1, 3)"), lit("_"), 
                                    col("language_full")))

In [0]:
usersDF = usersDF.withColumn("flag_long_title", length(col("civilitytitle")) > 10)

In [0]:
#casting datatypes based on original dataset schema

usersDF = usersDF.withColumn("hasanyapp", col("hasanyapp").cast("boolean"))
usersDF = usersDF.withColumn("hasandroidapp", col("hasandroidapp").cast("boolean"))
usersDF = usersDF.withColumn("hasiosapp", col("hasiosapp").cast("boolean"))
usersDF = usersDF.withColumn("hasprofilepicture", col("hasprofilepicture").cast("boolean"))


usersDF = usersDF.withColumn("socialnbfollowers", col("socialnbfollowers").cast(IntegerType()))
usersDF = usersDF.withColumn("socialnbfollows", col("socialnbfollows").cast(IntegerType()))

usersDF = usersDF.withColumn("productspassrate", col("productspassrate").cast(DecimalType(10, 2)))
usersDF = usersDF.withColumn("seniorityasmonths", col("seniorityasmonths").cast(DecimalType(10, 2)))
usersDF = usersDF.withColumn("seniorityasyears", col("seniorityasyears").cast(DecimalType(10, 2)))


In [0]:
usersDF = usersDF.withColumn("dayssincelastlogin",
                             when(col("dayssincelastlogin").isNotNull(),
                                  col("dayssincelastlogin").cast(IntegerType()))
                             .otherwise(0))

### Save the delta table after transformations

In [0]:
usersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/users")

### Read, Transform and load buyers data

In [0]:
buyersDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/buyers")

In [0]:
# Casting Integer columns
integer_columns = [
    'buyers', 'topbuyers', 'femalebuyers', 'malebuyers',
    'topfemalebuyers', 'topmalebuyers', 'totalproductsbought',
    'totalproductswished', 'totalproductsliked', 'toptotalproductsbought',
    'toptotalproductswished', 'toptotalproductsliked'
]

for column_name in integer_columns:
    buyersDF = buyersDF.withColumn(column_name, col(column_name).cast(IntegerType()))

In [0]:
# Casting Decimal columns
decimal_columns = [
    'topbuyerratio', 'femalebuyersratio', 'topfemalebuyersratio',
    'boughtperwishlistratio', 'boughtperlikeratio', 'topboughtperwishlistratio',
    'topboughtperlikeratio', 'meanproductsbought', 'meanproductswished',
    'meanproductsliked', 'topmeanproductsbought', 'topmeanproductswished',
    'topmeanproductsliked', 'meanofflinedays', 'topmeanofflinedays',
    'meanfollowers', 'meanfollowing', 'topmeanfollowers', 'topmeanfollowing'
]

for column_name in decimal_columns:
    buyersDF = buyersDF.withColumn(column_name, col(column_name).cast(DecimalType(10, 2)))

In [0]:
# Normalize country names
buyersDF = buyersDF.withColumn("country", initcap(col("country")))

for col_name in integer_columns:
    buyersDF = buyersDF.fillna({col_name: 0})

# Calculate the ratio of female to male buyers
buyersDF = buyersDF.withColumn("female_to_male_ratio", 
                               round(col("femalebuyers") / (col("malebuyers") + 1), 2))

# Determine the market potential by comparing wishlist and purchases
buyersDF = buyersDF.withColumn("wishlist_to_purchase_ratio", 
                               round(col("totalproductswished") / (col("totalproductsbought") + 1), 2))

# Tag countries with a high engagement ratio
high_engagement_threshold = 0.5
buyersDF = buyersDF.withColumn("high_engagement",
                               when(col("boughtperwishlistratio") > high_engagement_threshold, True)
                               .otherwise(False))
                               
    # Flag markets with increasing female buyer participation
buyersDF = buyersDF.withColumn("growing_female_market",
                               when(col("femalebuyersratio") > col("topfemalebuyersratio"), True)
                               .otherwise(False))

In [0]:
buyersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/buyers")

### Read, Transform and load sellers data

In [0]:
sellersDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/sellers")

In [0]:
sellersDF = sellersDF \
    .withColumn("nbsellers", col("nbsellers").cast(IntegerType())) \
    .withColumn("meanproductssold", col("meanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("meanproductslisted", col("meanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meansellerpassrate", col("meansellerpassrate").cast(DecimalType(10, 2))) \
    .withColumn("totalproductssold", col("totalproductssold").cast(IntegerType())) \
    .withColumn("totalproductslisted", col("totalproductslisted").cast(IntegerType())) \
    .withColumn("meanproductsbought", col("meanproductsbought").cast(DecimalType(10, 2))) \
    .withColumn("meanproductswished", col("meanproductswished").cast(DecimalType(10, 2))) \
    .withColumn("meanproductsliked", col("meanproductsliked").cast(DecimalType(10, 2))) \
    .withColumn("totalbought", col("totalbought").cast(IntegerType())) \
    .withColumn("totalwished", col("totalwished").cast(IntegerType())) \
    .withColumn("totalproductsliked", col("totalproductsliked").cast(IntegerType())) \
    .withColumn("meanfollowers", col("meanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("meanfollows", col("meanfollows").cast(DecimalType(10, 2))) \
    .withColumn("percentofappusers", col("percentofappusers").cast(DecimalType(10, 2))) \
    .withColumn("percentofiosusers", col("percentofiosusers").cast(DecimalType(10, 2))) \
    .withColumn("meanseniority", col("meanseniority").cast(DecimalType(10, 2)))

In [0]:
# Normalize country names and gender values
sellersDF = sellersDF.withColumn("country", initcap(col("country"))) \
                                                .withColumn("sex", upper(col("sex")))


#Add a column to categorize the number of sellers
sellersDF = sellersDF.withColumn("seller_size_category", 
                               when(col("nbsellers") < 500, "Small") \
                               .when((col("nbsellers") >= 500) & (col("nbsellers") < 2000), "Medium") \
                               .otherwise("Large"))

# Calculate the mean products listed per seller as an indicator of seller activity
sellersDF = sellersDF.withColumn("mean_products_listed_per_seller", 
                               round(col("totalproductslisted") / col("nbsellers"), 2))

# Identify markets with high seller pass rate
sellersDF = sellersDF.withColumn("high_seller_pass_rate", 
                               when(col("meansellerpassrate") > 0.75, "High") \
                               .otherwise("Normal"))

mean_pass_rate = sellersDF.select(round(avg("meansellerpassrate"), 2).alias("avg_pass_rate")).collect()[0]["avg_pass_rate"]

sellersDF = sellersDF.withColumn("meansellerpassrate",
                                 when(col("meansellerpassrate").isNull(), mean_pass_rate)
                                 .otherwise(col("meansellerpassrate")))

In [0]:
sellersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/sellers")

### Read, Transforma and load countries data

In [0]:
countriesDF = spark.read.format("delta").load("/mnt/delta/tables/bronze/countries")

In [0]:
countriesDF = countriesDF \
    .withColumn("sellers", col("sellers").cast(IntegerType())) \
    .withColumn("topsellers", col("topsellers").cast(IntegerType())) \
    .withColumn("topsellerratio", col("topsellerratio").cast(DecimalType(10, 2))) \
    .withColumn("femalesellersratio", col("femalesellersratio").cast(DecimalType(10, 2))) \
    .withColumn("topfemalesellersratio", col("topfemalesellersratio").cast(DecimalType(10, 2))) \
    .withColumn("femalesellers", col("femalesellers").cast(IntegerType())) \
    .withColumn("malesellers", col("malesellers").cast(IntegerType())) \
    .withColumn("topfemalesellers", col("topfemalesellers").cast(IntegerType())) \
    .withColumn("topmalesellers", col("topmalesellers").cast(IntegerType())) \
    .withColumn("countrysoldratio", col("countrysoldratio").cast(DecimalType(10, 2))) \
    .withColumn("bestsoldratio", col("bestsoldratio").cast(DecimalType(10, 2))) \
    .withColumn("toptotalproductssold", col("toptotalproductssold").cast(IntegerType())) \
    .withColumn("totalproductssold", col("totalproductssold").cast(IntegerType())) \
    .withColumn("toptotalproductslisted", col("toptotalproductslisted").cast(IntegerType())) \
    .withColumn("totalproductslisted", col("totalproductslisted").cast(IntegerType())) \
    .withColumn("topmeanproductssold", col("topmeanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("topmeanproductslisted", col("topmeanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meanproductssold", col("meanproductssold").cast(DecimalType(10, 2))) \
    .withColumn("meanproductslisted", col("meanproductslisted").cast(DecimalType(10, 2))) \
    .withColumn("meanofflinedays", col("meanofflinedays").cast(DecimalType(10, 2))) \
    .withColumn("topmeanofflinedays", col("topmeanofflinedays").cast(DecimalType(10, 2))) \
    .withColumn("meanfollowers", col("meanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("meanfollowing", col("meanfollowing").cast(DecimalType(10, 2))) \
    .withColumn("topmeanfollowers", col("topmeanfollowers").cast(DecimalType(10, 2))) \
    .withColumn("topmeanfollowing", col("topmeanfollowing").cast(DecimalType(10, 2)))


In [0]:
countriesDF = countriesDF.withColumn("country", initcap(col("country")))


# Calculating the ratio of top sellers to total sellers
countriesDF = countriesDF.withColumn("top_seller_ratio", 
                                        round(col("topsellers") / col("sellers"), 2))

# countriesDF countries with a high ratio of female sellers
countriesDF = countriesDF.withColumn("high_female_seller_ratio", 
                                        when(col("femalesellersratio") > 0.5, True).otherwise(False))

# Adding a performance indicator based on the sold/listed ratio
countriesDF = countriesDF.withColumn("performance_indicator", 
                                        round(col("toptotalproductssold") / (col("toptotalproductslisted") + 1), 2))

# Flag countries with exceptionally high performance
performance_threshold = 0.8
countriesDF = countriesDF.withColumn("high_performance", 
                                        when(col("performance_indicator") > performance_threshold, True).otherwise(False))

countriesDF = countriesDF.withColumn("activity_level",
                                       when(col("meanofflinedays") < 30, "Highly Active")
                                       .when((col("meanofflinedays") >= 30) & (col("meanofflinedays") < 60), "Moderately Active")
                                       .otherwise("Low Activity"))

In [0]:
countriesDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/silver/countries")

In [0]:
%fs ls "/mnt/delta/tables/silver/"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/silver/buyers/,buyers/,0,1731577557000
dbfs:/mnt/delta/tables/silver/countries/,countries/,0,1731577849000
dbfs:/mnt/delta/tables/silver/sellers/,sellers/,0,1731577718000
dbfs:/mnt/delta/tables/silver/users/,users/,0,1731577336000


In [0]:
%fs ls "/mnt/delta/tables/silver/users"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/silver/users/_delta_log/,_delta_log/,0,1731577336000
dbfs:/mnt/delta/tables/silver/users/part-00000-02850aa0-27e4-4151-922f-2cbfcc13e3cb.c000.snappy.parquet,part-00000-02850aa0-27e4-4151-922f-2cbfcc13e3cb.c000.snappy.parquet,1517594,1731660732000
dbfs:/mnt/delta/tables/silver/users/part-00000-19649c7f-eba0-43a5-9181-ddc5cd9d9adf.c000.snappy.parquet,part-00000-19649c7f-eba0-43a5-9181-ddc5cd9d9adf.c000.snappy.parquet,2648079,1731661097000
dbfs:/mnt/delta/tables/silver/users/part-00000-20659ec9-d55a-48ab-8884-b776c9995954.c000.snappy.parquet,part-00000-20659ec9-d55a-48ab-8884-b776c9995954.c000.snappy.parquet,2648079,1731661082000
dbfs:/mnt/delta/tables/silver/users/part-00000-31e5504f-465b-41fd-bbd8-e5dc42a03783.c000.snappy.parquet,part-00000-31e5504f-465b-41fd-bbd8-e5dc42a03783.c000.snappy.parquet,1517597,1731660755000
dbfs:/mnt/delta/tables/silver/users/part-00000-41b89153-d6d9-4c47-b8e5-bd297dfcc9aa.c000.snappy.parquet,part-00000-41b89153-d6d9-4c47-b8e5-bd297dfcc9aa.c000.snappy.parquet,1517595,1731660756000
dbfs:/mnt/delta/tables/silver/users/part-00000-7040b622-474b-4cb0-9f07-2b339980549f.c000.snappy.parquet,part-00000-7040b622-474b-4cb0-9f07-2b339980549f.c000.snappy.parquet,1517596,1731660752000
dbfs:/mnt/delta/tables/silver/users/part-00000-73ca5981-91a0-495e-813d-227231066fa8.c000.snappy.parquet,part-00000-73ca5981-91a0-495e-813d-227231066fa8.c000.snappy.parquet,2648079,1731661087000
dbfs:/mnt/delta/tables/silver/users/part-00000-9439ba4f-080c-428b-b742-7ba418dc55b8.c000.snappy.parquet,part-00000-9439ba4f-080c-428b-b742-7ba418dc55b8.c000.snappy.parquet,1019286,1731668855000
dbfs:/mnt/delta/tables/silver/users/part-00000-9c420487-288d-4685-bc07-2c53e70359c8.c000.snappy.parquet,part-00000-9c420487-288d-4685-bc07-2c53e70359c8.c000.snappy.parquet,2648079,1731661102000


In [0]:
%fs ls "/mnt/delta/tables/silver/countries"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/silver/countries/_delta_log/,_delta_log/,0,1731577849000
dbfs:/mnt/delta/tables/silver/countries/part-00000-05fee6c5-6a92-44e7-8a0e-1d3745885dff.c000.snappy.parquet,part-00000-05fee6c5-6a92-44e7-8a0e-1d3745885dff.c000.snappy.parquet,10249,1731660770000
dbfs:/mnt/delta/tables/silver/countries/part-00000-2dfb873f-0198-4cd9-955a-705aa910fe7a.c000.snappy.parquet,part-00000-2dfb873f-0198-4cd9-955a-705aa910fe7a.c000.snappy.parquet,10248,1731660769000
dbfs:/mnt/delta/tables/silver/countries/part-00000-4952c6d3-8a80-4cf7-951a-c2e005e0a6f7.c000.snappy.parquet,part-00000-4952c6d3-8a80-4cf7-951a-c2e005e0a6f7.c000.snappy.parquet,10248,1731661126000
dbfs:/mnt/delta/tables/silver/countries/part-00000-4bd4847a-b670-42de-bed9-8aa017bbc516.c000.snappy.parquet,part-00000-4bd4847a-b670-42de-bed9-8aa017bbc516.c000.snappy.parquet,10248,1731661120000
dbfs:/mnt/delta/tables/silver/countries/part-00000-4da480cb-4510-49f7-9453-ae9c25ab0ceb.c000.snappy.parquet,part-00000-4da480cb-4510-49f7-9453-ae9c25ab0ceb.c000.snappy.parquet,10217,1731668885000
dbfs:/mnt/delta/tables/silver/countries/part-00000-6e7be2f2-528e-4e62-aba2-64526ebabbda.c000.snappy.parquet,part-00000-6e7be2f2-528e-4e62-aba2-64526ebabbda.c000.snappy.parquet,10248,1731661133000
dbfs:/mnt/delta/tables/silver/countries/part-00000-74a4b945-161a-49bc-9eba-8ca3b838b474.c000.snappy.parquet,part-00000-74a4b945-161a-49bc-9eba-8ca3b838b474.c000.snappy.parquet,10246,1731660743000
dbfs:/mnt/delta/tables/silver/countries/part-00000-75419561-07c7-4305-81ee-6c5b723d6f45.c000.snappy.parquet,part-00000-75419561-07c7-4305-81ee-6c5b723d6f45.c000.snappy.parquet,10248,1731660235000
dbfs:/mnt/delta/tables/silver/countries/part-00000-8301689b-6240-43c3-9e06-5149cfd1f842.c000.snappy.parquet,part-00000-8301689b-6240-43c3-9e06-5149cfd1f842.c000.snappy.parquet,10248,1731661109000


In [0]:
%fs ls "/mnt/delta/tables/silver/sellers"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/silver/sellers/_delta_log/,_delta_log/,0,1731577718000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-01138934-2cf2-4b6c-951d-ebf4f634ca8b.c000.snappy.parquet,part-00000-01138934-2cf2-4b6c-951d-ebf4f634ca8b.c000.snappy.parquet,11539,1731661119000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-016f57ed-02bf-47ad-bbd7-b8ea38074bc3.c000.snappy.parquet,part-00000-016f57ed-02bf-47ad-bbd7-b8ea38074bc3.c000.snappy.parquet,11540,1731660766000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-5e007c0e-7f27-4606-848d-62d4cb8ac482.c000.snappy.parquet,part-00000-5e007c0e-7f27-4606-848d-62d4cb8ac482.c000.snappy.parquet,11508,1731577718000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-8fd48ea2-8acc-4f74-bb45-f464f32576a8.c000.snappy.parquet,part-00000-8fd48ea2-8acc-4f74-bb45-f464f32576a8.c000.snappy.parquet,11508,1731668878000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-ad96291b-e894-42cf-a6a8-0317693abba2.c000.snappy.parquet,part-00000-ad96291b-e894-42cf-a6a8-0317693abba2.c000.snappy.parquet,11539,1731661097000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-bc9f9df4-3caa-4d62-b79d-472c528a9844.c000.snappy.parquet,part-00000-bc9f9df4-3caa-4d62-b79d-472c528a9844.c000.snappy.parquet,11539,1731661125000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-c9c56025-d8b4-484d-95ff-f4c3b9055995.c000.snappy.parquet,part-00000-c9c56025-d8b4-484d-95ff-f4c3b9055995.c000.snappy.parquet,11539,1731660764000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-dc1ea2ec-2b45-4c3b-a98b-b01819a953d6.c000.snappy.parquet,part-00000-dc1ea2ec-2b45-4c3b-a98b-b01819a953d6.c000.snappy.parquet,11539,1731661110000
dbfs:/mnt/delta/tables/silver/sellers/part-00000-e36d620f-a34e-42c8-a159-faf3740be56c.c000.snappy.parquet,part-00000-e36d620f-a34e-42c8-a159-faf3740be56c.c000.snappy.parquet,11539,1731660232000


In [0]:
%fs ls "/mnt/delta/tables/silver/buyers"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/silver/buyers/_delta_log/,_delta_log/,0,1731577557000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-16d880e0-4b02-4ca4-8e0c-35321765898f.c000.snappy.parquet,part-00000-16d880e0-4b02-4ca4-8e0c-35321765898f.c000.snappy.parquet,16086,1731668865000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-195e9553-b5e2-4455-8df0-57006300c49d.c000.snappy.parquet,part-00000-195e9553-b5e2-4455-8df0-57006300c49d.c000.snappy.parquet,16115,1731660736000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-4368d5da-1b35-4370-aea8-3e15e44e50a8.c000.snappy.parquet,part-00000-4368d5da-1b35-4370-aea8-3e15e44e50a8.c000.snappy.parquet,16086,1731577558000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-453d0b85-0c3c-496a-aa36-fb749ba98016.c000.snappy.parquet,part-00000-453d0b85-0c3c-496a-aa36-fb749ba98016.c000.snappy.parquet,16117,1731661115000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-48ffa411-7537-4ca4-9aba-7cdd9867d9d1.c000.snappy.parquet,part-00000-48ffa411-7537-4ca4-9aba-7cdd9867d9d1.c000.snappy.parquet,16117,1731661110000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-4fdd104c-7ee6-4ae6-b05a-7e79e5790d88.c000.snappy.parquet,part-00000-4fdd104c-7ee6-4ae6-b05a-7e79e5790d88.c000.snappy.parquet,16117,1731660228000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-aa7f7fbc-2e1f-4583-b0dd-c09631e42a0d.c000.snappy.parquet,part-00000-aa7f7fbc-2e1f-4583-b0dd-c09631e42a0d.c000.snappy.parquet,16118,1731660761000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-d345fd0b-f7e2-4db8-8558-cbd8b59ed4ca.c000.snappy.parquet,part-00000-d345fd0b-f7e2-4db8-8558-cbd8b59ed4ca.c000.snappy.parquet,16117,1731660759000
dbfs:/mnt/delta/tables/silver/buyers/part-00000-e825e8d8-8f5b-4b36-b285-0bb632f4464d.c000.snappy.parquet,part-00000-e825e8d8-8f5b-4b36-b285-0bb632f4464d.c000.snappy.parquet,16117,1731661100000
